<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Predicting-gene-knockout-strategies" data-toc-modified-id="Predicting-gene-knockout-strategies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Predicting gene knockout strategies</a></span><ul class="toc-item"><li><span><a href="#Exercises" data-toc-modified-id="Exercises-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Exercises</a></span></li><li><span><a href="#Solutions" data-toc-modified-id="Solutions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Solutions</a></span><ul class="toc-item"><li><span><a href="#Solution-1" data-toc-modified-id="Solution-1-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Solution 1</a></span></li><li><span><a href="#Solution-2" data-toc-modified-id="Solution-2-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Solution 2</a></span></li></ul></li></ul></li><li><span><a href="#Predict-gene-knockout-strategies" data-toc-modified-id="Predict-gene-knockout-strategies-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Predict gene knockout strategies</a></span><ul class="toc-item"><li><span><a href="#OptGene" data-toc-modified-id="OptGene-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>OptGene</a></span></li><li><span><a href="#OptKnock" data-toc-modified-id="OptKnock-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>OptKnock</a></span></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Exercises</a></span></li><li><span><a href="#References" data-toc-modified-id="References-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>References</a></span></li></ul></li></ul></div>

# Predicting gene knockout strategies

In [7]:
%matplotlib inline
from cobra.io import read_sbml_model
from cameo import phenotypic_phase_plane

In [8]:
model = read_sbml_model('data/e_coli_core.xml.gz')

In [9]:
model.reactions.EX_o2_e.lower_bound = 0

In [10]:
model.reactions.EX_succ_e

Reaction identifier,EX_succ_e
Name,Succinate exchange
Memory address,0x012424f588
Stoichiometry,succ_e --> Succinate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
production_envelope = phenotypic_phase_plane(model, 
                                             variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                             objective=model.metabolites.succ_e)
production_envelope.plot()

In [6]:
mutant1 = model.copy()

In [27]:
mutant1.reactions.ACKr.knock_out()
mutant1.reactions.ATPS4r.knock_out()
mutant1.reactions.ATPM.knock_out()
mutant1.reactions.FUM.knock_out()

In [28]:
ppp = phenotypic_phase_plane(mutant1,
                             variables=[mutant1.reactions.BIOMASS_Ecoli_core_w_GAM],
                             objective=mutant1.reactions.EX_succ_e)

ppp.plot()

In [29]:
mutant2 = model.copy()

In [30]:
mutant2.reactions.ACALD.knock_out()
mutant2.reactions.PYK.knock_out()
mutant2.reactions.ME2.knock_out()

In [31]:
ppp = phenotypic_phase_plane(mutant2,
                             variables=[mutant2.reactions.BIOMASS_Ecoli_core_w_GAM],
                             objective=mutant2.reactions.EX_succ_e)

ppp.plot()

In [35]:
mutant3 = model.copy()

In [33]:
mutant3.reactions.ACALD.knock_out()
mutant3.reactions.LDH_D.knock_out()

In [34]:
ppp = phenotypic_phase_plane(mutant3,
                             variables=[mutant3.reactions.BIOMASS_Ecoli_core_w_GAM],
                             objective=mutant3.reactions.EX_succ_e)

ppp.plot()

We can also knock-out genes instead, and implicitly block the reactions that are annotated to only occur in presence of the corresponding gene. To reproduce the ACALD, LDH_D knock-out above with genes instead we can do:

In [36]:
mutant4 = model.copy()
mutant4.reactions.ACALD.gene_reaction_rule

'b0351 or b1241'

In [37]:
mutant4.reactions.LDH_D.gene_reaction_rule

'b2133 or b1380'

In [38]:
to_knock = mutant4.reactions.ACALD.genes.union(mutant4.reactions.LDH_D.genes)
for gene in to_knock:
    gene.knock_out()

In [39]:
ppp = phenotypic_phase_plane(mutant4,
                             variables=[mutant4.reactions.BIOMASS_Ecoli_core_w_GAM],
                             objective=mutant4.reactions.EX_succ_e)

ppp.plot()

## Exercises

* Do any of these desigs work under aerobic conditions?
* Did the gene-knockout design above have any unintended effects?

## Solutions

### Solution 1

### Solution 2

# Predict gene knockout strategies

In cameo we have two ways of predicting gene knockout targets: using evolutionary algorithms (OptGene) or linear programming (OptKnock)

## OptGene

OptGene is an approach to search for gene or reaction knockouts that relies on evolutionary algorithms[1](#references). The following image from authors summarizes the OptGene workflow.

<img src="http://static-content.springer.com/image/art%3A10.1186%2F1471-2105-6-308/MediaObjects/12859_2005_Article_632_Fig1_HTML.jpg"/>

In [40]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [41]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/e_coli_core.xml.gz')

In [42]:
optgene = OptGene(model)

In [43]:
result = optgene.run(target=model.reactions.EX_succ_e, 
                     biomass=model.reactions.BIOMASS_Ecoli_core_w_GAM,
                     substrate=model.metabolites.glc__D_e,
                     max_evaluations=20000, population_size=200, max_knockouts=5,
                     plot=False, growth_coupled=True)

Starting optimization at Thu, 28 Feb 2019 11:46:46


HBox()

Finished after 00:02:39


In [ ]:
result

<div class="alert alert-warning">
The optimization above proposes to knock out ATPS4r (ATP synthase) and `s0001` (an spontaneous reaction) which probably isn't a good strategy.
</div>

<div class="alert alert-success">
Workaround: Use the `essential_genes` parameter to `OptGene` to create an exclude list.
</div>

In [ ]:
essential = [g.id for g in model.reactions.ATPS4r.genes] + ['s0001']
optgene = OptGene(model, essential_genes=essential)
result = optgene.run(target=model.reactions.EX_succ_e, 
                     biomass=model.reactions.BIOMASS_Ecoli_core_w_GAM,
                     substrate=model.metabolites.glc__D_e,
                     max_evaluations=20000, population_size=200, max_knockouts=5,
                     plot=False, growth_coupled=True)
result

In [ ]:
result.plot(0)

In [ ]:


result.display_on_map(0, "e_coli_core.Core metabolism")

## OptKnock

OptKnock uses a bi-level mixed integer linear programming approach to identify reaction knockouts[[2](#references)]:

$$
\begin{matrix}
maximize & \mathit{v_{chemical}} & & (\mathbf{OptKnock}) \\
\mathit{y_j} & & & \\
subject~to & maximize & \mathit{v_{biomass}} & (\mathbf{Primal}) \\
& \mathit{v_j} & & & & \\
\end{matrix}\\
\begin{bmatrix}
subject~to  & \sum_{j=1}^{M}S_{ij}v_{j} = 0,\\ 
& v_{carbon\_uptake} = v_{carbon~target}\\ 
& v_{apt} \ge v_{apt\_main}\\ 
& v_{biomass} \ge v_{target\_biomass}\\ 
& v_{j}^{min} \cdot y_j \le v_j \le v_{j}^{max} \cdot y_j, \forall j \in \boldsymbol{M} \\
\end{bmatrix}\\
\begin{align}
 & y_j = {0, 1}, & & \forall j \in \boldsymbol{M} & \\
 & \sum_{j \in M} (1 - y_j) \le K& & & \\
\end{align}
$$



In [ ]:
from cameo.strain_design.deterministic.linear_programming import OptKnock

In [ ]:
optknock = OptKnock(model, fraction_of_optimum=0.1, exclude_reactions=['ATPS4r', 'CO2t', 'H2Ot'])

Again, we include a couple of reactions on the exclude list to avoid uninteresting solution. Running multiple knockouts with OptKnock can take a few hours or days...

In [ ]:
result = optknock.run(max_knockouts=3, target="EX_ac_e", biomass="BIOMASS_Ecoli_core_w_GAM")

In [ ]:
result

In [ ]:
result.plot(0)

In [ ]:
result.display_on_map(0, "e_coli_core.Core metabolism")

## Exercises

* Use OptGene or OptKnock to find a growth coupled design for a product of your choice!

<a id="references"></a>
## References

[1]Patil, K. R., Rocha, I., Förster, J., & Nielsen, J. (2005). Evolutionary programming as a platform for in silico metabolic engineering. BMC Bioinformatics, 6, 308. doi:10.1186/1471-2105-6-308

[2]Burgard, A.P., Pharkya, P., Maranas, C.D. (2003), "OptKnock: A Bilevel Programming Framework for Identifying Gene Knockout Strategies for Microbial Strain Optimization," Biotechnology and Bioengineering, 84(6), 647-657.